### 마켓 데이터 크롤링 100 - KRX 04 
# 상세 가격 데이터 구축

<img src="http://i.imgur.com/grQHNKG.jpg">
    
#### 2017 FinanceData http://fb.com/financedata

# 시가총액 순위 정보
* 시장정보 → 주식 → 순위정보 → 시가총액 상/하위 정보로 가격 데이터 구축
* http://marketdata.krx.co.kr/contents/MKD/04/0404/04040200/MKD04040200.jsp
* 일별: 종목코드, 종목명, 현재가, 등락률, 거래량, 거래대금, 시가총액, 시가총액비중(%), 상장주식수(천주), 외국인, 보유주식수, 외국인, 지분율(%)
* 1995-05-02 부터 현재까지 일자별

In [1]:
import requests

# STEP 01: Generate OTP
gen_otp_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'
gen_otp_data = {
    'name':'fileDown',
    'filetype':'xls',
    'url':'MKD/04/0404/04040200/mkd04040200_01',
    'market_gubun':'ALL', #시장구분: ALL=전체, STK=코스피, KSQ=코스피, KNX=코넥스
    'indx_ind_cd':'', 
    'sect_tp_cd':'',
    'schdate':'20160601',
    'pagePath':'/contents/MKD/04/0404/04040200/MKD04040200.jsp',
}
    
r = requests.post(gen_otp_url, gen_otp_data)
code = r.content

# STEP 02: download
down_url = 'http://file.krx.co.kr/download.jspx'
down_data = {
    'code': code,
}

r = requests.post(down_url, down_data)
with open("marcap_data.xls", 'wb') as f:
    f.write(r.content)

# stock_master_krx()

In [2]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from datetime import datetime

def stock_master_krx(date_str=None):
    # 시가총액순위 정보를 DataFrame으로 반환
    if date_str == None:
        date_str = datetime.today().strftime('%Y%m%d')

    # STEP 01: Generate OTP
    gen_otp_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'
    gen_otp_data = {
        'name':'fileDown',
        'filetype':'xls',
        'url':'MKD/04/0404/04040200/mkd04040200_01',
        'market_gubun':'ALL', #시장구분: ALL=전체
        'indx_ind_cd':'',
        'sect_tp_cd':'',
        'schdate': date_str,
        'pagePath':'/contents/MKD/04/0404/04040200/MKD04040200.jsp',
    }
    
    r = requests.post(gen_otp_url, gen_otp_data)
    code = r.content
    
    # STEP 02: download
    down_url = 'http://file.krx.co.kr/download.jspx'
    down_data = {
        'code': code,
    }
    
    r = requests.post(down_url, down_data)
    df = pd.read_excel(BytesIO(r.content), header=0, thousands=',', converters={'종목코드': str})
    return df

In [12]:
# 1996-03-02, 1997-01-08 날짜의 데이터는 거래소 xlsx 파일 오류
    
date_str = datetime(2006, 1, 3).strftime('%Y%m%d')
df = stock_master_krx(date_str)
print('종목수:', len(df))
df.head(10)

종목수: 1787


,종목코드,종목명,현재가,대비,등락률,거래량,거래대금,시가,고가,저가,시가총액,시가총액비중(%),상장주식수(천주),외국인 보유주식수,외국인 지분율(%)
0,005930,삼성전자,665000,6000,0.9,358018,236617710000,660000,666000,656000,97954059105000,13.30,147299337,79309412,53.84
1,060000,국민은행,75100,-300,-0.4,956964,71309108700,75100,75900,74000,25262071611600,3.43,336379116,287389119,85.44
2,015760,한국전력,37300,100,0.3,1155189,42680099200,37000,37300,36650,23899921772900,3.24,640748573,192066477,29.98
3,005380,현대차,97000,-1400,-1.4,1231153,119324527000,98400,98900,95600,21248694094000,2.88,219058702,98428967,44.93
4,005490,POSCO,200500,-1000,-0.5,207084,41393114500,199000,201500,199000,17480960417500,2.37,87186835,59288792,68.00
5,000660,하이닉스,38100,500,1.3,13834043,520842229050,38100,38400,36850,17060138727000,2.32,447772670,84043632,18.77
6,053000,우리금융,20250,50,0.2,4324750,88552156450,20350,20900,20100,16321810635000,2.22,806015340,92496112,11.48
7,034220,LG필립스LCD,43300,-400,-0.9,903718,39173709500,44200,44400,42600,15493419810000,2.10,357815700,193301274,54.02
8,055550,신한지주,41400,-500,-1.2,1079858,44679754570,41900,42000,40900,14871182758200,2.02,359207313,205178878,57.12
9,017670,SK텔레콤,177000,-3000,-1.7,361744,64082216500,179500,180000,176000,14562977847000,1.98,82276711,40299364,48.98


# 날짜 생성

In [4]:
start = datetime(2017, 1, 1)
end = datetime(2017, 1, 31)
dates = pd.date_range(start=start, end=end)
dates

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10', '2017-01-11', '2017-01-12',
               '2017-01-13', '2017-01-14', '2017-01-15', '2017-01-16',
               '2017-01-17', '2017-01-18', '2017-01-19', '2017-01-20',
               '2017-01-21', '2017-01-22', '2017-01-23', '2017-01-24',
               '2017-01-25', '2017-01-26', '2017-01-27', '2017-01-28',
               '2017-01-29', '2017-01-30', '2017-01-31'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
# 날짜 xlsx 파일을 marcap_daily에 저장

start = datetime(2017, 1, 20)
end = datetime(2017, 1, 31)
dates = pd.date_range(start=start, end=end)

for date in dates:
    date_str = date.strftime('%Y%m%d')
    df = stock_master_krx(date_str)
    print(date_str, end=', ')
    print('count: ', len(df))

20170120, count:  2244
20170121, count:  0
20170122, count:  0
20170123, count:  2244
20170124, count:  2245
20170125, count:  2246


In [ ]:
%%file krx_marcap_xlsx.py
#!/usr/bin/python3

import pandas as pd
import numpy as np
import requests
from io import BytesIO
import os
from datetime import datetime, timedelta

def stock_master_krx(date_str=None):
    # 시가총액순위 정보를 DataFrame으로 반환
    if date_str == None:
        date_str = datetime.today().strftime('%Y%m%d')

    # STEP 01: Generate OTP
    gen_otp_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'
    gen_otp_data = {
        'name':'fileDown',
        'filetype':'xls',
        'url':'MKD/04/0404/04040200/mkd04040200_01',
        'market_gubun':'ALL', #시장구분: ALL=전체
        'indx_ind_cd':'',
        'sect_tp_cd':'',
        'schdate': date_str,
        'pagePath':'/contents/MKD/04/0404/04040200/MKD04040200.jsp',
    }
    
    r = requests.post(gen_otp_url, gen_otp_data)
    code = r.content
    
    # STEP 02: download
    down_url = 'http://file.krx.co.kr/download.jspx'
    down_data = {
        'code': code,
    }
    
    r = requests.post(down_url, down_data)
    df = pd.read_excel(BytesIO(r.content), header=0, thousands=',', converters={'종목코드': str})
    return df

if __name__ == "__main__":
    start = datetime(1996, 1, 1)
    end = datetime.today() - timedelta(days=1)
    dates = pd.date_range(start=start, end=end)

    data_dir = './krx_marcap_xlsx'
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    for date in dates:
        date_str = date.strftime('%Y%m%d')
        fname = data_dir + '/' + 'marcap_' + date_str + '.xlsx'
        if not os.path.isfile(fname):
            print(date_str, end=', ')
            df = stock_master_krx(date_str)
            print('count: ', len(df))
            df.to_excel(fname, index=False)

다음과 같이 실행

```bash
$ python3 krx_marcap_xlsx.py
```

In [ ]:
%%file krx_marcap_xlsx2sqlite.py
#!/usr/bin/python3

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import sqlite3

create_table_query = """
    CREATE TABLE IF NOT EXISTS "marcap" (
        "날짜" TIMESTAMP,
        "종목코드" TEXT,
        "종목명" TEXT,
        "현재가" INTEGER,
        "대비" INTEGER,
        "등락률" REAL,
        "거래량" INTEGER,
        "거래대금" INTEGER,
        "시가총액" INTEGER,
        "시가총액비중(%)" REAL,
        "상장주식수(천주)" INTEGER,
        "외국인 보유주식수" REAL,
        "외국인 지분율(%)" REAL
    );

    CREATE INDEX IF NOT EXISTS "ix_marcap_날짜" ON "marcap" ("날짜");
    CREATE INDEX IF NOT EXISTS "ix_marcap_종목코드" ON "marcap" ("종목코드");
"""

if __name__ == "__main__":
    start = datetime(2000, 1, 1)
    end = datetime.today() - timedelta(days=1) # yearterday
    dates = pd.date_range(start=start, end=end)
    
    data_dir = './krx_marcap_xlsx'
    if not os.path.exists(data_dir):
        print(data_dir, ' not exists')
        exit(1)

    data_dir_to = './krx_marcap_sqlite'
    if not os.path.exists(data_dir_to):
        os.makedirs(data_dir_to)

    db_fname = data_dir_to + '/' + 'marcap.db'
    con = sqlite3.connect(db_fname)
    con.executescript(create_table_query)

    # db에 저장된 가장 최근 날짜
    latest_date = str(datetime(1970, 1, 1))
    df = pd.read_sql('select * from marcap order by "날짜" desc limit 1', con)
    if len(df) > 0:
        latest_date = df['날짜'][0]

    for date in dates:
        # skip previous date
        if str(date) < latest_date:
            continue
            
        # read .xlsx and write to sqlite
        date_str = date.strftime('%Y%m%d')
        fname = data_dir + '/' + 'marcap_' + date_str + '.xlsx'
        df = pd.read_excel(fname)
        df['날짜'] = date
        df.set_index('날짜', inplace=True)
        print(date_str, end=', ')
        print('count: ', len(df))
        df.to_sql('marcap', con, if_exists='append')
    con.close()

다음과 같이 실행

```bash
$ python3 krx_marcap_xlsx2sqlite.py
```

----
#### 2017 FinanceData http://fb.com/financedata http://financedata.github.com